# Multiclass SVM 구현

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris')
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [32]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [33]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [34]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [35]:
!pip install cvxopt

In [36]:
from itertools import combinations
from collections import Counter
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

## convex optimization 경험으로 cvxopt 사용법을 알아 참고해보았습니다
## sklearn의 SVM 없이 구현하기 위해 라그랑주 승수법 사용
## 라그랑주 승수법을 직접 풀기 위해 cvxopt를 사용해 최적화 진행

## 참고 1. https://leechanwoo-kor.github.io/machine%20learning/svm-python/
## 참고 2. https://sanghyu.tistory.com/7

In [37]:
## One Vs one 알고리즘 만들기

In [38]:
## 한 클래스에 대해 판별 가능한 SVM Class 형성

class SVM_ovo:
    ## 불러오자마자 세팅할 것들 - 커널 trick 없이 선형 kernel을 사용했다고 가정
    def __init__(self):
        self.labels_map = None
        self.X = None
        self.y = None
        self.alphas = None
        self.w = None
        self.b = None

    ## y 값들 음성(-1) , 양성(+1) 샘플로 만들기  -------------ㄱ
    def make_label_map(self, uniq_labels):
        labels_map = list(zip([-1, 1], uniq_labels))
        self.labels_map = labels_map
        return

    def transform_label(self, label, labels_map):
        res = [l[0] for l in labels_map if l[1] == label][0]
        return res

    ## --------------------------------------------------------⨼



    def inverse_label(self, svm_label, labels_map): ##
        try:
            res = [l[1] for l in labels_map if l[0] == svm_label][0]
        except:
            print(svm_label)
            print(labels_map)
            raise
        return res

    def fit(self, X, y, C):

        uniq_labels = np.unique(y)
        self.make_label_map(uniq_labels)
        self.X = X
        y = [self.transform_label(label, self.labels_map) for label in y] ## 1, -1로 변환
        y = np.array(y)

        ## formulating standard form
        m, n = X.shape
        y = y.reshape(-1,1)*1.
        self.y = y
        yX = y*X

        ## Quadratic Programming 을 위한 P/q/G/h/A/b 설정 - 제약조건이 있는 QP 문제에서 기본 형태
        ## 1/2 w*w을 최소화 하고 라그랑지안에서 Duality Gap을 없애기 위한 조건을 합쳐 QP문제로 변형
        ## 풀려는 x : 라그랑지안

        ## cvxopt에선 각 행렬을 cxopt_matrix()로 묶어서 진행
        Q = np.dot(yX,yX.T)
        P = cvxopt_matrix(Q)
        q = cvxopt_matrix(-np.ones((m, 1)))

        ## 부등 제약조건
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1, np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))

        ## 등호 제약조건
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))

        ## cvxopt 문제풀기
        cvxopt_solvers.options['show_progress'] = False ## 결과 출력 표시 X
        sol = cvxopt_solvers.qp(P, q, G, h, A, b) ## 주어진 행렬을 이용해 최적화 진행
        alphas = np.array(sol['x']) ## 풀었던 x값 즉
        S = (alphas>1e-4).flatten()

        w = ((y*alphas).T@X).reshape(-1,1)
        b = np.mean(y[S] - np.dot(X[S],w))
        self.w = w      ## 결정경계 가중치
        self.b = b   ## 결정경계 Bias
        self.alphas = alphas    ##라그랑주 승수법의 Alpha 변수
        return

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions

    def _predict(self, x):

        res = np.sign(self.w.T.dot(x)+self.b)
        res = self.inverse_label(res, self.labels_map)
        return res

In [53]:
## SVM 클래스를 이용해 전체 데이터셋에 대해 예측을 진행

class SVM():
    def __init__(self,  kernel=None):

        self.X = None
        self.y = None
        self.model_list = None
        if kernel is not None:
            assert kernel in ['poly', 'rbf', 'sigmoid']

    def fit(self, X, y, C):
        self._fit_svc(X, y, C)

    def _fit_svc(self, X, y, C):
        uniq_labels = np.unique(y) ## 직접 y의 클래스 수를 받아오고 조합(Combination)을 이용해 다수의 multi class에서도 대응이 가능
        label_combinations = list(combinations(uniq_labels, 2))

        model_list = [] ## 가능한 ovo 모델을 저장
        for lc in label_combinations: ## 가능한 조합에 대해 SVM 초평면 찾기
            target_idx = np.array([x in lc for x in y])
            y_restricted = y[target_idx]
            X_restricted = X[target_idx]
            clf = SVM_ovo()
            clf.fit(X_restricted, y_restricted, C)
            model_list.append(clf)
        self.model_list = model_list
        return


    def predict(self, X):

        model_list = self.model_list
        prediction = [model.predict(X) for model in model_list]
        prediction = [Counter(pred).most_common(1)[0][0] for pred in list(zip(*prediction))]
        ## 선택한 방법 : 제일 많이 나온 라벨 값을 정답으로 선정

        return prediction


## 평가를 위한 함수 설정 _ accuracy

def acc_count(result, y_test):
    acc_score = sum(result==y_test)/len(y_test)

    return acc_score

In [54]:
for i in [0.01,0.1,1,3,5,10,20,30,50,100]:

    classifier = SVM()
    classifier.fit(X_train,y_train,i)
    result = classifier.predict(X_test)
    accuracy_score = acc_count(result,y_test)
    print('C : ',i, ' Acc : ', accuracy_score)

C :  0.01  Acc :  0.8333333333333334
C :  0.1  Acc :  0.9666666666666667
C :  1  Acc :  0.9333333333333333
C :  3  Acc :  0.9333333333333333
C :  5  Acc :  0.9333333333333333
C :  10  Acc :  0.9333333333333333
C :  20  Acc :  0.9333333333333333
C :  30  Acc :  0.9333333333333333
C :  50  Acc :  0.9333333333333333
C :  100  Acc :  0.9333333333333333


In [45]:
## 하이퍼파라미터 C : 0.1 일때 0.9666667로 나오는 것을 확인